In [2]:
import random
from random import randint
from numpy import array
from numpy import zeros
import os
import tensorflow as tf

#import keras
import keras.backend.tensorflow_backend as KTF
from keras import backend as K
from keras.callbacks import LearningRateScheduler
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D,AveragePooling2D
from keras.layers import LSTM,Bidirectional,BatchNormalization,Activation,GRU
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.optimizers import Adam,SGD
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras import regularizers
import numpy as np
import cv2
from keras.preprocessing.image import img_to_array,array_to_img
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from hilbert import hilbertCurve
from PIL import Image,ImageChops, ImageEnhance
import patch_hilbert_data_set as dt
os.environ["CUDA_VISIBLE_DEVICES"] = "7"
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
session = tf.Session(config=config)
np.random.seed(2020)

In [10]:
batch_size=64
model = Sequential()
model.add(TimeDistributed(Conv2D(64, kernel_size=3, strides=1, padding = 'same', 
    activation= 'relu'), input_shape=(16,64,64,3),name='block1_conv1'))# 卷积层
model.add(TimeDistributed(Conv2D(64, kernel_size=3, strides=1, padding = 'same', 
    activation= 'relu'), name='block1_conv2'))# 卷积层
model.add(TimeDistributed(BatchNormalization(momentum=0.1)))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2),name='block1_pool')))
#model.add(Dropout(0.5))
model.add(TimeDistributed(Conv2D(128, kernel_size=3, strides=1, padding = 'same', 
    activation= 'relu'), name='block2_conv1'))
model.add(TimeDistributed(Conv2D(128, kernel_size=3, strides=1, padding = 'same', 
    activation= 'relu'), name='block2_conv2'))
model.add(TimeDistributed(BatchNormalization(momentum=0.1)))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2),name='block2_pool')))
#model.add(Dropout(0.5))
model.add(TimeDistributed(Conv2D(256, kernel_size=3, strides=1, padding = 'same', 
    activation= 'relu'), name='block3_conv1'))
model.add(TimeDistributed(Conv2D(256, kernel_size=3, strides=1, padding = 'same', 
    activation= 'relu'), name='block3_conv2'))
#model.add(TimeDistributed(Conv2D(256, kernel_size=3, strides=1, padding = 'same', 
#    activation= 'relu'), name='block3_conv3'))
model.add(TimeDistributed(BatchNormalization(momentum=0.1)))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2),name='block3_pool')))
#model.add(Dropout(0.25))
#model.add(Dropout(0.5))
model.add(TimeDistributed(Flatten()))
model.add(Dropout(0.5))#wu
model.add(GRU(256,return_sequences=True))
model.add(Dropout(0.5))
model.add(GRU(256))
model.add(Dropout(0.5))

model.add(Dense(256,kernel_regularizer=regularizers.l2(0.1)))
model.add(BatchNormalization(momentum=0.1))
model.add(Activation('relu'))
#model.add(Dropout(0.5))
model.add(Dense(256,kernel_regularizer=regularizers.l2(0.1)))#
model.add(BatchNormalization(momentum=0.1))
model.add(Activation('relu'))

model.add(Dense(1, activation='sigmoid'))#softmax

In [11]:
sgd = SGD(lr=0.01, decay=0.000015, momentum=0.9, nesterov=True)
def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer.lr
    return lr
lr_metric = get_lr_metric(sgd)
def scheduler(epoch):
    # 每隔100个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        lr = K.get_value(model.optimizer.lr)
        K.set_value(model.optimizer.lr, lr * 0.8)
        print("lr changed to {}".format(lr * 0.8))
    return K.get_value(model.optimizer.lr)
reduce_lr = LearningRateScheduler(scheduler)
metric=['acc', lr_metric]
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['acc'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (TimeDistribute (None, 16, 64, 64, 64)    1792      
_________________________________________________________________
block1_conv2 (TimeDistribute (None, 16, 64, 64, 64)    36928     
_________________________________________________________________
time_distributed_8 (TimeDist (None, 16, 64, 64, 64)    256       
_________________________________________________________________
time_distributed_9 (TimeDist (None, 16, 32, 32, 64)    0         
_________________________________________________________________
block2_conv1 (TimeDistribute (None, 16, 32, 32, 128)   73856     
_________________________________________________________________
block2_conv2 (TimeDistribute (None, 16, 32, 32, 128)   147584    
_________________________________________________________________
time_distributed_10 (TimeDis (None, 16, 32, 32, 128)  

In [6]:
train_datagen = ImageDataGenerator(#用于数据增强
#    rotation_range=15,
    shear_range=0.1,
    rescale=1./255,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)
validate_datagen = ImageDataGenerator(
    rescale=1./255
)
def data_generator(generator,images,labels,batch_size):#自定义generator，输出多标签
    #input_generator=generator.flow(images,batch_size=batch_size)
    num_samples=len(images)
    input_generator=generator.flow(images,labels,batch_size=batch_size)#届时传入之前的数据增强generator
    while True:
        for offset in range(0,num_samples,batch_size):
        	#使用了flow(),每一次返回的是批量的 图片数据，组合标签[labels1,labels2]
            batch_samples,batch_labels=input_generator.next()
            X_train=[]
            Y_train=[]
            for i in range(len(batch_samples)):
                img=batch_samples[i]
                img = dt.read_image1(img)
                X_train.append(img) 
                Y_train.append(batch_labels[i])
                #print(Y_train)
            X_train=np.array(X_train)
            Y_train=np.array(Y_train)
            yield X_train,Y_train#返回batch的图片数据和对应的标签

In [6]:
#
fileList = os.listdir('/home/Data/FF/RAW1/originalt')  # 训练数据文件夹
data=[]
labels=[]
for fileName in fileList:
    image=cv2.imread(os.path.join('/home/Data/FF/RAW1/originalt',fileName))
    if image is not None:
        image2=img_to_array(image)
        data.append(image2)
        labels.append('0')

fileList = os.listdir('/home/Data/FF/RAW1/train/fake/DF')  # 训练数据文件夹  
for fileName in fileList:
    image=cv2.imread(os.path.join('/home/Data/FF/RAW1/train/fake/DF',fileName))
    if image is not None:
        image2=img_to_array(image)
        data.append(image2)
        labels.append('1')

data=np.array(data,dtype="float32")
labels=np.array(labels,dtype="float32")

fileList = os.listdir('/home/Data/FF/RAW1/original')  # 训练数据文件夹
datat=[]
labelts=[]
    
for fileName in fileList:
    image=cv2.imread(os.path.join('/home/Data/FF/RAW1/original',fileName))
    if image is not None:
        image2=img_to_array(image)
        datat.append(image2)
        labelts.append('0')
fileList = os.listdir('/home/Data/FF/RAW1/val/fake/DF')  # 训练数据文件夹  
for fileName in fileList:
    image=cv2.imread(os.path.join('/home/Data/FF/RAW1/val/fake/DF',fileName))
    if image is not None:
        image2=img_to_array(image)
        datat.append(image2)
        labelts.append('1')

datat=np.array(datat,dtype="float32")
labelts=np.array(labelts,dtype="float32")

print(data.shape)
print(datat.shape)

(14000, 160, 160, 3)
(6000, 160, 160, 3)


In [7]:
#
index = [i for i in range(len(data))]
random.shuffle(index);
data=data[index]
labels=labels[index]

indext = [i for i in range(len(datat))]
random.shuffle(indext);
datat=datat[indext]
labelts=labelts[indext]

In [8]:
#
train_datagen = ImageDataGenerator(#用于数据增强
#    rotation_range=15,
    shear_range=0.1,
    rescale=1./255,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)
validate_datagen = ImageDataGenerator(
    rescale=1./255
)
def data_generator(generator,images,labels,batch_size):#自定义generator，输出多标签
    #input_generator=generator.flow(images,batch_size=batch_size)
    num_samples=len(images)
    input_generator=generator.flow(images,labels,batch_size=batch_size)#届时传入之前的数据增强generator
    while True:
        for offset in range(0,num_samples,batch_size):
        	#使用了flow(),每一次返回的是批量的 图片数据，组合标签[labels1,labels2]
            batch_samples,batch_labels=input_generator.next()
            X_train=[]
            Y_train=[]
            for i in range(len(batch_samples)):
                img=batch_samples[i]
                img = dt.read_image1(img)
                X_train.append(img) 
                Y_train.append(batch_labels[i])
                #print(Y_train)
            X_train=np.array(X_train)
            Y_train=np.array(Y_train)
            yield X_train,Y_train#返回batch的图片数据和对应的标签

In [ ]:
total_validate=len(labelts)
total_train=data.shape[0]
filepath='./model/weights.best.FF.hdf5'

checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1,save_best_only=True,mode='max') 
callbacks_list = [checkpoint,reduce_lr]
history=model.fit_generator(
    data_generator(train_datagen,data,labels,batch_size),
    epochs=50,
    callbacks=callbacks_list,
    validation_data=data_generator(validate_datagen,datat,labelts,batch_size),
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size,
)